In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt 

In [2]:
#read csv, change for location
df = pd.read_csv('data/PHY_TRAIN.csv')

In [3]:
#splitting target and predictors
y = df['target']
X = df.drop(['target', 'exampleid'],axis=1)

In [5]:
#STEP 1: Data Exploration
X.describe()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,0.155606,0.084876,-0.050354,-0.000060,0.126569,0.049887,-0.038344,0.002860,0.848353,0.673485,...,0.008120,0.000478,0.003198,0.052807,0.066944,-0.014101,-0.001460,0.094480,0.002843,0.066545
std,0.414875,0.295335,0.253748,0.392916,0.400694,0.223713,0.214168,0.322077,0.453585,0.511087,...,0.769302,0.446978,0.381329,0.180710,0.283114,0.176896,0.295939,0.315841,0.019081,0.223091
min,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,...,-1.000000,-0.999998,-0.908001,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.522596,0.250301,...,-1.000000,-0.000013,-0.001582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.787572,0.599672,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.105687,1.018601,...,1.000000,0.000015,0.003002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.639020,3.429590,0.999954,1.000000,2.719006,3.054644,0.999274,1.000000,6.699783,5.283748,...,1.000000,0.999999,0.907744,0.999953,3.429590,0.999869,1.000000,3.000000,0.385513,1.000000


In [6]:
#Data exploration for target variables
y.describe()

count    50000.000000
mean         0.497220
std          0.499997
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: target, dtype: float64

In [8]:
y.value_counts()

#sns.countplot(x = y, data=y)
#plt.show

0    25139
1    24861
Name: target, dtype: int64

In [9]:
y

0        0
1        0
2        1
3        0
4        0
        ..
49995    0
49996    1
49997    1
49998    1
49999    0
Name: target, Length: 50000, dtype: int64

In [10]:
#STEP 2: Missing value indicator
from sklearn.impute import MissingIndicator
#True indicates Missing Value, False indicates no missing value

#indicator = MissingIndicator(features='all')
indicator = MissingIndicator(features='missing-only')
missing = indicator.fit_transform(X)
#converts boolean to integers
missing.astype(int)

array([[1, 1, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [11]:
#STEP 3: Replace missing values with mean
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer()
mean_imputer = mean_imputer.fit(X)
df1 = pd.DataFrame(mean_imputer.transform(X))

In [144]:
y = y.to_numpy()

In [145]:
#STEP 4 Ignored
#STEP 5 Modeling
#Logistic Regression w/o Interaction Terms

import statsmodels.api as sm
logit_model=sm.Logit(y,df1)
result=logit_model.fit()
#print(result.summary2())

#******Might want to try Recursive Feature Elimination

Optimization terminated successfully.
         Current function value: 0.541341
         Iterations 7


LinAlgError: Singular matrix

In [63]:
#Logistic regression with at least three two-way interactions

In [64]:
#Random Forest

In [65]:
#Gradient Boosting

In [ ]:
#Step 6: Comparisons of fitted models using c-statistics, i.e., AUC of the ROC curve